# Exercise 2 : Graph-based SLAM

The goal of this assignment is to implement a least-squares based method to address the SLAM problem in its graph-based formulation as explained in the lecture. 

We will consider 2D poses $(x, y, \theta)$ of the robot and 2D landmarks $(x_l, y_l)$ as the unknowns in our SLAM problem. The data is provided as a graph structure containing nodes (poses and landmarks) and constraints between these nodes (i.e pose-pose and pose-landmark). These datasets are stored as `dataset.g2o` text files. You are provided with the following datasets (see `data` folder), each of which represents the measurements of one SLAM problem.
1. `simulation-pose-pose.g2o`: simulated measurements containing pose-pose constraints only  
2. `simulation-pose-landmark.g2o`: simulated measurements containing both pose-pose and pose-landmark constraints
3. `intel.g2o`: real world measurements containing pose-pose constraints only
4. `dlr.g2o`: real world measurements containing both pose-pose and pose-landmark constraints

To get started with this task, we provide some Python code which will help in loading the graph structure, visualize it and other functions that you may need. We also provide additional notes `graph-slam-notes.pdf` which explains the Jacobian computations in detail.

## Task 2.0 Understanding the graph structure

Each graph consists of a set of nodes (or vertices) and edges that connect these nodes. As discussed in the lecture, the nodes correspond to the unknowns of the least-squares SLAM, whereas the edges correspond to the constraints obtained from the measurements.  In this assignment, the graph has the following types of nodes and edges:

1. Nodes:

    `VERTEX_SE2`: These nodes represent the 2D pose of the robot $(x, y, \theta)$

    `VERTEX_XY`: These nodes represent the 2D location of the landmark $(x_l, y_l)$
    
2. Edges:

    `EDGE_SE2`: These edges represent a constraint between two VERTEX_SE2 nodes. We refer to these edges as pose-pose constraints.
    
    `EDGE_SE2_XY`: These edges represent a constraint between a VERTEX_SE2 node and a VERTEX_XY node. We refer these edges as pose-landmark edge. 
    

In our code, we represent the graph as a class with the following attributes:
1. `nodes`: A dictionary of nodes where the information of each `node` can be accessed with `nodeId` as a key. Each `node` has a unique `nodeId`. This node can be either `VERTEX_SE2` or `VERTEX_XY`. If node has a dimension of 3, it represents the pose the robot (`VERTEX_SE2`). If the node has a dimension of 2, it represents the location of landmark (`VERTEX_XY`).
  
    
2. `edges`: A list of all the `edges` in the graph where each `edge` has the following attributes:

`Type`: The type is 'P' if the constraint is a pose-pose constraint (`EDGE_SE2`), whereas it is 'L' of it is a pose-landmark constraint (`EDGE_SE2_XY`).
`fromNode`: `nodeId` of the node from which the edge originates from.
`toNode`: `nodeId` of the node to which the edge terminates to.
`measurement`: The measurement corresponding to the edge.
`information`: The corresponding information matrix for the edge constraint.
    
3. `x`: All the unknowns (node variables) are stacked into a vector. This should be used for updating the state after each iteration of the optimization.

4. `lut`: This is a lookup table (implemented as a dictionary in Python). `lut[nodeId]` provides the starting location of the variables of the node with id `nodeId`.

Go through the examples in the cell below to understand how to work with the graph structure. Ensure that you understand how the graph is organized. This will be neccessary to solve all the tasks in the assignment. 

In [ ]:
import ex2 as ex
%matplotlib inline

# load a dataset 
filename = 'data/simulation-pose-landmark.g2o'
graph = ex.read_graph_g2o(filename)

# visualize the dataset
ex.plot_graph(graph)
print('Loaded graph with {} nodes and {} edges'.format(len(graph.nodes), len(graph.edges)))

# print information for the two types of nodes
nodeId = 128
print('Node {} = {} is a VERTEX_SE2 node'.format(nodeId, graph.nodes[nodeId]))

# access the state vector using the lookup table
fromIdx = graph.lut[nodeId]
print('Node {} from the state vector = {}'.format(nodeId,graph.x[fromIdx:fromIdx+3]))

nodeId = 1
print('Node {} = {} is a VERTEX_XY node'.format(nodeId, graph.nodes[nodeId]))

# access the state vector using the lookup table
fromIdx = graph.lut[nodeId]
print('Node {} from the state vector = {}'.format(nodeId, graph.x[fromIdx:fromIdx+2]))

# print information for two types of edges
eid = 0 
print('Edge {} = {} is a pose-pose constraint'.format(eid, graph.edges[eid]))

eid = 1 
print('Edge {} = {} is a pose-landmark constraint'.format(eid, graph.edges[eid]))

## Task 1.2.1 Computing total error. [5]

In this task, you have implement the function  `compute_global_error` function  for computing the current error value for a graph with constraints. Compute this error for all the four datasets and print the results. 

## 1.2.2 Linearization of a Pose-Pose constraint [5]

Since the constraint described by the edge between two pose nodes is non-linear, you need to linearize it solve the least-squares optimization. The function `linearize_pose_pose_constraint` computes the error and the Jacobian for pose-pose constraint. It takes as input: 
   
    x1 : 3x1 vector 
         $(x,y,\theta)$ of the first robot pose
    x2 : 3x1 vector
         $(x,y,\theta)$ of the second robot pose
    z :  3x1 vector
         $(x,y,\theta)$ of the measurement
    
and returns

    e  : 3x1
         error of the constraint
    A  : 3x3
         Jacobian wrt x1
    B  : 3x3
         Jacobian wrt x2

Implement the `linearize_pose_pose_constraint` function and verify that it is correct. 

**Hint** : You can verify the Jacobian computation by comparing against a numerically Jacobian for your test data.


## 1.2.3 Linearization of a Pose-Landmark constraint [5]

Since the constraint described by the edge between a pose and landmark node is non-linear, you need to linearize it solve the least-squares optimization. The function `linearize_pose_landmark_constraint` computes the error and the Jacobian for pose-landmark constraint. It takes as input: 

    x : 3x1 vector
        (x,y,theta) og the robot pose
    l : 2x1 vector
        (x,y) of the landmark
    z : 2x1 vector
        (x,y) of the measurement
    
and returns:

    e : 2x1 vector
        error for the constraint
    A : 2x3 Jacobian wrt x
    B : 2x2 Jacobian wrt l
    
Implement the `linearize_and_solve` function and verify that it is correct. 

**Hint** : You can verify the Jacobian computation by comparing against a numerically Jacobian for your test data.

## 1.2.4 Building and Solving Linearized System [5]

The `linearize_and_solve` function builds the $H$ and $b$ matrices in order to solve for $dx$ (i.e. change in the unknowns $x$) for one iteration. The function takes as input:

    g : graph at iteration i
    
and returns

    dx : Nx1 vector 
        change in the solution for the unknowns x

Implement the `linearize_and_solve` function. Some skeletal code for the function is already provided to you to start with.

## 1.2.5 Iterative procedure for solving non-linear least squares [5]

The `run_graph_slam` function iteratively solves the least squares problem and updates the unknowns $x$. The procedure should be terminated if the change in $|dx| < 10^-4$ or the until a maximum number of iterations  (maxIter = 100) is reached. 

Implement the function `run_graph_slam` to perform the optimization. Some hints are provided as comments in the function. Test the function on the `simulation-pose-pose.g2o` and `simulation-pose-landmark.g2o` datasets. 

## 1.2.6 Results for different datasets [5]

Evaluate the results of the graph-slam algorithm for all the four datasets. 

1. Create a `gif` animation visualing the results after each iteration. 
2. Plot the error vs iterations for all the datasets.

Please run your code locally and upload the `gifs` generated from your results in the cell below.